In [14]:
import h5py
import lightning.pytorch as pl
import numpy as np
import onnxruntime as ort
import pandas as pd
import torch
import torch.nn.functional as F
from h5py import File
from sklearn.metrics import accuracy_score, f1_score
from torch import utils
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [2]:
f: File = h5py.File("./data/fault-data-2280.h5", "r")

codeArr = f['label']
waveArr = f['wave_data']
fftAbsArr = f['wave_fft_abs']

In [11]:
# 项目中 脚本，勿动
def get_vibration_pct(waveArr):
    fftArr = np.fft.fft(waveArr)

    fftArr = fftArr[:1024]
    fftAbsArr = np.abs(fftArr)

    # fftAbsArr = fftAbsArr / 10
    # print(fftAbsArr.max(), fftAbsArr.min())

    inputs = np.array(fftAbsArr, np.float32)

    result = sess.run(None, {"input": [inputs]})[0].squeeze()
    # print(result)

    b = F.softmax(torch.tensor(result), dim=0).numpy().tolist()
    # print(b)
    # round(random.random(), 3)
    return [round(a, 6) for a in b]


def vb_pprint(res):
    for i, pct in enumerate(res):
        print("{}: {:>5.2f}".format(i, pct * 100))

In [5]:
class MyDataModule(pl.LightningDataModule):
    def __init__(self, trainSet, batchSize: int, transform=None):
        super().__init__()
        self.batchSize = batchSize
        self.transform = transform
        self.trainSet, self.validSet, self.testsSet = utils.data.random_split(trainSet, [0.7, 0.2, 0.1])
        print('dataset num', len(self.trainSet), len(self.validSet), len(self.testsSet))

    def train_dataloader(self):
        return DataLoader(self.trainSet, batch_size=self.batchSize, shuffle=False)

    def val_dataloader(self):
        return DataLoader(self.validSet, batch_size=self.batchSize, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.testsSet, batch_size=self.batchSize, shuffle=False)

    def apply_transform(self, x):
        if self.transform:
            x = self.transform(x)
        return x


dataset = TensorDataset(torch.tensor(waveArr), torch.tensor(codeArr, dtype=torch.int64))
dm = MyDataModule(dataset, batchSize=128, transform=None)  # custom_transform

In [29]:
sess = ort.InferenceSession("vb-2024-05-14.onnx")

df = pd.DataFrame(columns=['id', "y", "y_", 'score'])


def get_onnx(dataSet, label: str = ''):
    y = dataSet[:][1].numpy().tolist()
    y_ = []
    for idx, one in enumerate(dataSet):
        # p_i, y_i = torch.max(torch.tensor(get_vibration_pct(one[0])), dim=-1)
        p_i, y_i = np.amax(get_vibration_pct(one[0])), np.argmax(get_vibration_pct(one[0]))
        y_.append(y_i)

        if label == '':
            df.loc[len(df)] = {'id': idx, 'y': y[idx], "y_": y_i.item(), 'score': p_i.item()}

    acc = accuracy_score(y, y_)
    f1 = f1_score(y, y_, average='macro')

    print('acc {:.3f} , f1 {:.3f} {}'.format(acc, f1, label))


get_onnx(dm.trainSet, 'train')
get_onnx(dm.validSet, 'valid')
get_onnx(dm.testsSet, 'tests')
print('------ onnx')
get_onnx(dataset)

In [31]:
df["ok"] = df["y"] == df["y_"]

pd.set_option('display.max_rows', None)  # 最多显示10行
df

In [10]:
"""
0:  0.09
1: 43.08
2:  6.83
3:  0.00
4: 48.29
5:  1.71
"""
# 2023-12-08 [4:34 PM] Edward Hui 完美多故障 wave_id: 1731903766443438080
w1 = [
    -0.44033495,
    -0.4349283,
    -0.51031786,
    -0.57725865,
    -0.48809773,
    -0.4472884,
    -0.4344096,
    -0.3864599,
    -0.3658239,
    -0.3321923,
    -0.26593375,
    -0.17405781,
    -0.13231386,
    -0.04562497,
    0.088247254,
    0.17211366,
    0.24039093,
    0.30658868,
    0.37231448,
    0.43945622,
    0.49509767,
    0.51491123,
    0.52070576,
    0.6020394,
    0.70199966,
    0.68567216,
    0.6398487,
    0.62239504,
    0.6037824,
    0.6308578,
    0.6360028,
    0.5620478,
    0.5203459,
    0.5008174,
    0.43762907,
    0.3670854,
    0.29762587,
    0.281733,
    0.22505413,
    0.17362304,
    0.1127852,
    0.08513036,
    0.0047827237,
    -0.03872762,
    -0.08574738,
    -0.15319753,
    -0.22446084,
    -0.28840154,
    -0.41018006,
    -0.5209256,
    -0.6075818,
    -0.65291923,
    -0.6670177,
    -0.6722374,
    -0.62129235,
    -0.62215215,
    -0.6252177,
    -0.5304024,
    -0.4796723,
    -0.46305978,
    -0.4013995,
    -0.3451038,
    -0.32821557,
    -0.27709287,
    -0.26451316,
    -0.2637748,
    -0.23399843,
    -0.18471694,
    -0.12836051,
    -0.10834603,
    -0.07232185,
    -0.0147083765,
    0.036447,
    0.0675178,
    0.13255201,
    0.24158254,
    0.30569616,
    0.34037918,
    0.42872232,
    0.47440562,
    0.52239734,
    0.5363136,
    0.543052,
    0.5602394,
    0.57130504,
    0.54058003,
    0.49605095,
    0.47184014,
    0.5197758,
    0.6340635,
    0.64943767,
    0.38517928,
    0.04680705,
    -0.2264749,
    -0.46099898,
    -0.6187315,
    -0.68180305,
    -0.72571063,
    -0.78688955,
    -0.74574375,
    -0.7349912,
    -0.75087005,
    -0.736823,
    -0.7368324,
    -0.7532346,
    -0.7447905,
    -0.74135584,
    -0.75186074,
    -0.7642863,
    -0.8280821,
    -0.8698167,
    -0.8791861,
    -0.84512454,
    -0.80693674,
    -0.8025348,
    -0.7543094,
    -0.65682584,
    -0.56546867,
    -0.44050318,
    -0.308874,
    -0.14516462,
    -0.008745623,
    0.1488608,
    0.29818198,
    0.37240794,
    0.41867068,
    0.5200048,
    0.5981561,
    0.6481152,
    0.66954565,
    0.77433777,
    1.034096,
    1.2602694,
    1.3685709,
    1.2188479,
    0.9287572,
    0.56505257,
    0.19772172,
    -0.00860076,
    -0.12128558,
    -0.21225962,
    -0.19932008,
    -0.098401874,
    -0.04524646,
    -0.164609,
    -0.2907614,
    -0.43751246,
    -0.59959096,
    -0.7068176,
    -0.76737046,
    -0.83037657,
    -0.87671405,
    -0.8948173,
    -0.98548293,
    -1.0742279,
    -1.0822514,
    -0.99849725,
    -0.94258475,
    -0.9148505,
    -0.88291514,
    -0.81013775,
    -0.78366053,
    -0.7785857,
    -0.7030091,
    -0.5730156,
    -0.4006845,
    -0.28405097,
    -0.20387158,
    -0.11490693,
    -0.01238122,
    0.09242492,
    0.16094986,
    0.21300712,
    0.2655317,
    0.33941185,
    0.3958057,
    0.43433928,
    0.489191,
    0.52053285,
    0.5228927,
    0.5385519,
    0.62775964,
    0.7028034,
    0.6836955,
    0.7740527,
    0.9536596,
    1.0543629,
    0.9775153,
    0.7951093,
    0.5446783,
    0.31637394,
    0.1474402,
    0.041414402,
    -0.044419337,
    -0.17760463,
    -0.30074763,
    -0.38516545,
    -0.4501576,
    -0.55471605,
    -0.6455359,
    -0.61377347,
    -0.5926562,
    -0.5717585,
    -0.58776355,
    -0.670228,
    -0.71741605,
    -0.7551272,
    -0.77761835,
    -0.765319,
    -0.7705761,
    -0.7401315,
    -0.67171407,
    -0.6412227,
    -0.6148389,
    -0.47686383,
    -0.32037425,
    -0.23045163,
    -0.1406692,
    -0.046110965,
    0.027671099,
    0.1586741,
    0.27800393,
    0.3004484,
    0.33193973,
    0.4154463,
    0.48368153,
    0.542767,
    0.5743565,
    0.61811924,
    0.6384001,
    0.66995215,
    0.7145046,
    0.7307433,
    0.80428237,
    0.8488114,
    0.8947003,
    1.0408438,
    1.2139786,
    1.2431568,
    1.016404,
    0.7088596,
    0.49160227,
    0.29316783,
    0.16749206,
    0.10758415,
    0.05707364,
    0.09124733,
    0.16551538,
    0.1901094,
    0.06262517,
    -0.05905986,
    -0.16369309,
    -0.31069648,
    -0.39045063,
    -0.35411334,
    -0.34815994,
    -0.41090906,
    -0.4432369,
    -0.43119922,
    -0.38335702,
    -0.36651084,
    -0.37583348,
    -0.3809878,
    -0.35625827,
    -0.27635455,
    -0.22740483,
    -0.18946938,
    -0.10309357,
    0.0077968114,
    0.11468244,
    0.2035069,
    0.2751347,
    0.32312644,
    0.4357505,
    0.5133925,
    0.5755575,
    0.6485825,
    0.6829011,
    0.70644367,
    0.7228739,
    0.7364397,
    0.7215375,
    0.74839324,
    0.80798805,
    0.7825155,
    0.7109344,
    0.69033116,
    0.67635417,
    0.70408845,
    0.7562439,
    0.67870003,
    0.5957121,
    0.4734336,
    0.37706226,
    0.3279303,
    0.25840068,
    0.17294078,
    0.07846666,
    0.06615796,
    0.04952674,
    0.012993192,
    -0.023367453,
    -0.1032104,
    -0.18469824,
    -0.21903081,
    -0.27680317,
    -0.35034692,
    -0.45135856,
    -0.53675306,
    -0.54601026,
    -0.5451738,
    -0.5420569,
    -0.534024,
    -0.50242513,
    -0.4257505,
    -0.37583348,
    -0.37045485,
    -0.29187825,
    -0.18197855,
    -0.124467894,
    -0.11005635,
    -0.0729527,
    -0.024788046,
    0.022652293,
    0.063022375,
    0.081270464,
    0.06615796,
    0.09115854,
    0.15509458,
    0.19957224,
    0.26309705,
    0.33097243,
    0.3691462,
    0.37296402,
    0.46780732,
    0.5464213,
    0.5880858,
    0.5581318,
    0.50590163,
    0.48787788,
    0.4828404,
    0.4782889,
    0.43838608,
    0.32420123,
    0.19383846,
    0.15420203,
    0.1210424,
    0.05624652,
    0.13443056,
    0.20451628,
    0.17520252,
    -0.0113438135,
    -0.25916722,
    -0.44155928,
    -0.58028674,
    -0.62025493,
    -0.6363347,
    -0.664569,
    -0.67138225,
    -0.68497604,
    -0.68732655,
    -0.65535384,
    -0.5831887,
    -0.56792665,
    -0.57300156,
    -0.56475836,
    -0.6040396,
    -0.64922285,
    -0.6540314,
    -0.60017973,
    -0.6064509,
    -0.6060023,
    -0.52030873,
    -0.4409144,
    -0.36091724,
    -0.2789761,
    -0.1476974,
    -0.037634134,
    0.12145362,
    0.29892966,
    0.43604958,
    0.5190515,
    0.5966421,
    0.645246,
    0.6044133,
    0.58643156,
    0.61548364,
    0.6704428,
    0.6765738,
    0.5955392,
    0.5478933,
    0.5860063,
    0.72195804,
    0.7750013,
    0.73011243,
    0.5341126,
    0.22892337,
    -0.08929419,
    -0.29255584,
    -0.39979196,
    -0.42766643,
    -0.464471,
    -0.50742996,
    -0.5140189,
    -0.60641354,
    -0.61770815,
    -0.5520338,
    -0.5367297,
    -0.6543725,
    -0.774665,
    -0.7844783,
    -0.8352318,
    -0.9279349,
    -0.97185177,
    -0.9823894,
    -0.9942822,
    -0.9994692,
    -0.9504914,
    -0.9220562,
    -0.8840647,
    -0.79986185,
    -0.68691534,
    -0.64949393,
    -0.60296017,
    -0.4922754,
    -0.38568416,
    -0.22876,
    -0.07637334,
    0.04189572,
    0.13850541,
    0.24522749,
    0.38347363,
    0.43458694,
    0.46454558,
    0.49764445,
    0.54776245,
    0.73964536,
    1.0723212,
    1.3158127,
    1.3840058,
    1.2779052,
    0.978749,
    0.6235446,
    0.3040793,
    0.15659928,
    0.04029288,
    -0.021255255,
    -0.0072642807,
    0.002698564,
    -0.032451775,
    -0.09686913,
    -0.1662025,
    -0.24917635,
    -0.31787887,
    -0.44755942,
    -0.543613,
    -0.61610997,
    -0.6516528,
    -0.74058944,
    -0.8414656,
    -0.92406094,
    -1.0215445,
    -1.0640454,
    -1.111841,
    -1.1175233,
    -1.0085535,
    -0.8676905,
    -0.79132426,
    -0.65772307,
    -0.57461375,
    -0.5041168,
    -0.42860103,
    -0.33713633,
    -0.21473165,
    -0.10807499,
    -0.011652231,
    0.03610587,
    0.06794772,
    0.15600581,
    0.22331576,
    0.25889602,
    0.36438906,
    0.37081444,
    0.40478253,
    0.51454204,
    0.66995215,
    0.88553655,
    1.1056163,
    1.2535262,
    1.2912841,
    1.1485893,
    0.9732722,
    0.7454305,
    0.6094274,
    0.54776245,
    0.47370467,
    0.37740338,
    0.30452323,
    0.29325196,
    0.2741674,
    0.23673664,
    0.23998438,
    0.19862361,
    0.07809749,
    -0.06241975,
    -0.139286,
    -0.19663776,
    -0.2897614,
    -0.39368904,
    -0.47336376,
    -0.5389961,
    -0.575796,
    -0.5825298,
    -0.57064635,
    -0.53737456,
    -0.48479858,
    -0.424386,
    -0.31079927,
    -0.26719546,
    -0.22480197,
    -0.101523444,
    -0.004016543,
    0.07416749,
    0.15697312,
    0.15283285,
    0.18724017,
    0.23500295,
    0.30226618,
    0.35013175,
    0.36284697,
    0.3316173,
    0.32601437,
    0.38261846,
    0.39641318,
    0.44046092,
    0.483775,
    0.52165437,
    0.5408137,
    0.59506726,
    0.63511026,
    0.7166402,
    0.8786532,
    0.9134344,
    0.8717699,
    0.8730456,
    0.8547788,
    0.85978365,
    0.78476787,
    0.7240936,
    0.7701601,
    0.7175327,
    0.6136191,
    0.44630682,
    0.1984881,
    0.0020443434,
    -0.16679597,
    -0.24580245,
    -0.26789173,
    -0.30232242,
    -0.3085095,
    -0.28979877,
    -0.2987242,
    -0.33802888,
    -0.41529232,
    -0.47684047,
    -0.49225202,
    -0.4883968,
    -0.45187727,
    -0.45108753,
    -0.4034042,
    -0.37045485,
    -0.38733375,
    -0.36878657,
    -0.29085487,
    -0.2180121,
    -0.15455736,
    -0.064438485,
    -0.005273581,
    0.052821208,
    0.12926689,
    0.20732474,
    0.2538632,
    0.3507813,
    0.3866466,
    0.3678144,
    0.4164557,
    0.49002746,
    0.53617805,
    0.5516737,
    0.6151612,
    0.67438686,
    0.7113877,
    0.7143457,
    0.67278403,
    0.63682055,
    0.61885756,
    0.56501514,
    0.5401314,
    0.49626124,
    0.4413114,
    0.43851694,
    0.36577696,
    0.2531576,
    0.15940776,
    0.1118506,
    0.1358418,
    0.11843486,
    -0.020661784,
    -0.19017968,
    -0.37084737,
    -0.4959811,
    -0.60570323,
    -0.6698028,
    -0.616853,
    -0.5734268,
    -0.5978853,
    -0.5786278,
    -0.54854304,
    -0.53052396,
    -0.53251463,
    -0.554445,
    -0.49456048,
    -0.40665194,
    -0.363679,
    -0.32631832,
    -0.30550942,
    -0.29505125,
    -0.2822285,
    -0.26784033,
    -0.24784455,
    -0.23157783,
    -0.17867942,
    -0.17405781,
    -0.08189683,
    -0.010128832,
    0.0030210014,
    0.054910038,
    0.12172466,
    0.20046477,
    0.3151169,
    0.42863354,
    0.52513105,
    0.6105162,
    0.6534238,
    0.65746593,
    0.64561987,
    0.66526514,
    0.65592384,
    0.57561356,
    0.5555664,
    0.48544323,
    0.39853472,
    0.38122123,
    0.38347363,
    0.28106475,
    0.07649465,
    -0.11857056,
    -0.3181873,
    -0.48397148,
    -0.6430452,
    -0.6825928,
    -0.71181774,
    -0.7682116,
    -0.77391267,
    -0.7773707,
    -0.7585945,
    -0.7652956,
    -0.749599,
    -0.7329678,
    -0.76313204,
    -0.7978758,
    -0.82156324,
    -0.84071326,
    -0.8829385,
    -0.899009,
    -0.9343883,
    -0.9146682,
    -0.8634521,
    -0.7681742,
    -0.6813778,
    -0.6078668,
    -0.50728977,
    -0.3264585,
    -0.14432348,
    -0.006465197,
    0.13982788,
    0.24690977,
    0.33481362,
    0.40639937,
    0.46869987,
    0.5357435,
    0.57398266,
    0.7263694,
    1.0217967,
    1.2556103,
    1.4257544,
    1.2930598,
    1.0415214,
    0.7038408,
    0.2742048,
    -0.016002798,
    -0.11477608,
    -0.1324821,
    -0.09983181,
    -0.030339574,
    0.009712744,
    -0.03535838,
    -0.14751515,
    -0.25899902,
    -0.4094838,
    -0.5719034,
    -0.6811208,
    -0.7221124,
    -0.8008292,
    -0.84311986,
    -0.90884566,
    -1.0125116,
    -1.0713774,
    -1.0958732,
    -1.058321,
    -1.0233856,
    -1.039157,
    -0.9737677,
    -0.8478816,
    -0.77373505,
    -0.6799806,
    -0.60193676,
    -0.47338244,
    -0.38446918,
    -0.30320096,
    -0.2107035,
    -0.058601905,
    0.049349163,
    0.12936035,
    0.2716907,
    0.38201097,
    0.43083453,
    0.48302263,
    0.5306312,
    0.5611319,
    0.6043899,
    0.6653306,
    0.7983196,
    0.9095418,
    1.0120956,
    0.9826556,
    0.8488114,
    0.6901349,
    0.5534728,
    0.415708,
    0.31093925,
    0.21599784,
    0.1269257,
    0.10497661,
    0.1656322,
    0.25291458,
    0.30793452,
    0.30479428,
    0.11852365,
    -0.14099164,
    -0.35179555,
    -0.5046729,
    -0.56733316,
    -0.57541746,
    -0.63142806,
    -0.74074835,
    -0.8146799,
    -0.84589094,
    -0.84451705,
    -0.8981304,
    -0.9331406,
    -0.849905,
    -0.7798567,
    -0.7703612,
    -0.662312,
    -0.5310473,
    -0.44220883,
    -0.3754129,
    -0.30956092,
    -0.16825394,
    -0.054008342,
    0.034605835,
    0.11099544,
    0.14892621,
    0.18367,
    0.30273348,
    0.39175418,
    0.48227027,
    0.5624637,
    0.57916033,
    0.64863396,
    0.68223286,
    0.6680222,
    0.7088082,
    0.7767584,
    0.785595,
    0.81559104,
    0.7952681,
    0.76787966,
    0.8855739,
    0.9998429,
    1.008016,
    0.80453,
    0.52468246,
    0.3269583,
    0.26948974,
    0.24039093,
    0.15723014,
    0.06440091,
    0.10420089,
    0.12482286,
    0.12823415,
    0.09461189,
    0.01113801,
    -0.10452818,
    -0.19516577,
    -0.253083,
    -0.29881302,
    -0.31901908,
    -0.33137918,
    -0.35072076,
    -0.3733708,
    -0.3741652,
    -0.36095464,
    -0.3606509,
    -0.25926068,
    -0.21602139,
    -0.18905349,
    -0.14065517,
    -0.09727101,
    -0.0060072425,
    0.072055295,
    0.14154755,
    0.20665184,
    0.25867173,
    0.3112991,
    0.39172614,
    0.47943842,
    0.5603936,
    0.5982729,
    0.6185772,
    0.6171519,
    0.6222081,
    0.6897237,
    0.7774453,
    0.842615,
    0.8722746,
    0.86088645,
    0.85978365,
    0.8282969,
    0.8098199,
    0.79055774,
    0.75913644,
    0.7446501,
    0.62668014,
    0.5232899,
    0.43098405,
    0.3507813,
    0.27384028,
    0.17747359,
    0.08468643,
    -0.009871817,
    -0.092453144,
    -0.107532926,
    -0.1838571,
    -0.28547624,
    -0.28452763,
    -0.28925672,
    -0.38824964,
    -0.46420932,
    -0.49423805,
    -0.49168193,
    -0.4994204,
    -0.49856994,
    -0.45087725,
    -0.4554381,
    -0.42157283,
    -0.33425307,
    -0.2836865,
    -0.2658029,
    -0.21378303,
    -0.1785953,
    -0.14982362,
    -0.15658545,
    -0.10121503,
    -0.10560297,
    -0.118893,
    -0.09727101,
    -0.097434565,
    -0.052830745,
    0.012091302,
    0.06393361,
    0.089130454,
    0.13183236,
    0.20969395,
    0.3320332,
    0.466083,
    0.5540243,
    0.6380029,
    0.6748729,
    0.66358286,
    0.6407506,
    0.65037227,
    0.6205912,
    0.5469914,
    0.4251942,
    0.28850886,
    0.15086551,
    0.037311506,
    -0.055961657,
    -0.18796468,
    -0.31144416,
    -0.39283386,
    -0.44757342,
    -0.43652645,
    -0.26844782,
    -0.096504636,
    -0.051433515,
    -0.1506741,
    -0.36174437,
    -0.5879598,
    -0.7386128,
    -0.79850197,
    -0.79913753,
    -0.7994459,
    -0.79639447,
    -0.84313387,
    -0.8052919,
    -0.7530757,
    -0.7346127,
    -0.71656555,
    -0.6711953,
    -0.63365245,
    -0.5697538,
    -0.467901,
    -0.36815572,
    -0.26610664,
    -0.17844577,
    -0.11950516,
    -0.020647764,
    0.076429226,
    0.19398332,
    0.2725552,
    0.34041655,
    0.42530635,
    0.47360653,
    0.49155086,
    0.55342144,
    0.60872644,
    0.62055385,
    0.6552229,
    0.69543874,
    0.76161784,
    0.8117171,
    0.84702635,
    0.8768915,
    0.79670274,
    0.59186625,
    0.3721042,
    0.1982638,
    0.046928547,
    -0.12264542,
    -0.25508302,
    -0.33894944,
    -0.40817067,
    -0.4394424,
    -0.5171638,
    -0.5695949,
    -0.6112034,
    -0.62682986,
    -0.6582932,
    -0.6815367,
    -0.7480195,
    -0.81519866,
    -0.85461545,
    -0.9300424,
    -0.98000616,
    -1.0085769,
    -1.0220118,
    -1.0379934,
    -1.010161,
    -0.90962136,
    -0.8478069,
    -0.8213156,
    -0.8251241,
    -0.737552,
    -0.6269233,
    -0.55228144,
    -0.43743768,
    -0.30193457,
    -0.15373024,
    -0.06329827,
    0.029689835,
    0.17116037,
    0.26777005,
    0.427269,
    0.5128925,
    0.5777304,
    0.72685534,
    0.98029107,
    1.2185628,
    1.3938706,
    1.3136257,
    1.114238,
    0.80829644,
    0.4512649,
    0.24582095,
    0.20209566,
    0.15720677,
    0.11033187,
    0.08966785,
    0.088111736,
    0.056101654,
    -0.0038576608,
    -0.063050605,
    -0.11282276,
    -0.19417977,
    -0.35059458,
    -0.48225647,
    -0.57441276,
    -0.63625526,
    -0.68943876,
    -0.7915813,
    -0.87520003,
    -0.9530523,
    -1.0355074,
    -1.1203551,
    -1.162445,
    -1.1648328,
    -1.0849618,
    -1.0087124,
    -0.90033144,
    -0.8081424,
    -0.7155795,
    -0.6439938,
    -0.5773708,
    -0.43806854,
    -0.3188088,
    -0.24919038,
    -0.17760463,
    -0.04064355,
    0.12382751,
    0.23468518,
    0.29489684,
    0.4335028,
    0.54043984,
    0.59098774,
    0.7326592,
    1.0067917,
    1.2905505,
    1.4752649,
    1.386076,
    1.1293085,
    0.87983084,
    0.68687314,
    0.5429773,
    0.4772141,
    0.39704403,
    0.366291,
    0.44936764,
    0.5000417,
    0.50441563,
    0.5372295,
    0.45097983,
    0.29543892,
    0.18609996,
    0.05178847,
    -0.100079484,
    -0.24469027,
    -0.28241077,
    -0.3480338,
    -0.4379564,
    -0.5294538,
    -0.58103913,
    -0.6394236,
    -0.64795184,
    -0.58525413,
    -0.5067103,
    -0.49754187,
    -0.45627457,
    -0.35744053,
    -0.272018,
    -0.23961072,
    -0.17446904,
    -0.04678855,
    0.073116064,
    0.11046739,
    0.13282304,
    0.18782897,
    0.20727335,
    0.27282625,
    0.35167384,
    0.40867513,
    0.4262129,
    0.46387732,
    0.49896693,
    0.5503419,
    0.51603276,
    0.52671057,
    0.5365986,
    0.57341254,
    0.65651727,
    0.7269909,
    0.7955766,
    0.81587607,
    0.8904946,
    0.91517276,
    0.88878894,
    0.87707376,
    0.8845646,
    0.83535314,
    0.7400052,
    0.75414103,
    0.77717423,
    0.75065494,
    0.64039075,
    0.40213293,
    0.2309281,
    0.0074136253,
    -0.13592143,
    -0.19994159,
    -0.25582603,
    -0.32740712,
    -0.3736465,
    -0.45422313,
    -0.50391585,
    -0.56886125,
    -0.5798475,
    -0.5485804,
    -0.5321735,
    -0.47847134,
    -0.4339423,
    -0.3856468,
    -0.32551455,
    -0.27530313,
    -0.22085328,
    -0.19696021,
    -0.16129585,
    -0.10592541,
    0.00866599,
    0.034531068,
    0.034839485,
    0.06624675,
    0.11041599,
    0.14722991,
    0.1535151,
    0.15922084,
    0.17853437,
    0.20046477,
    0.30287835,
    0.36130488,
    0.44632086,
    0.5126822,
    0.56176275,
    0.57652944,
    0.6224698,
    0.6734149,
    0.6940041,
    0.6618352,
    0.61937624,
    0.5649124,
    0.5529635,
    0.5379818,
    0.6544191,
    0.8122545,
    0.818292,
    0.626311,
    0.30848125,
    -0.0020959382,
    -0.30169156,
    -0.47832647,
    -0.50081766,
    -0.55920213,
    -0.64057314,
    -0.6459845,
    -0.62220824,
    -0.6099837,
    -0.5994648,
    -0.5939506,
    -0.6020443,
    -0.56254804,
    -0.5148553,
    -0.538954,
    -0.57199216,
    -0.58071667,
    -0.5531413,
    -0.50608414,
    -0.4654944,
    -0.404736,
    -0.38376355,
    -0.339842,
    -0.28292945,
    -0.21711488,
    -0.18080564,
    -0.09711213,
    -0.0149046425,
    0.05977931,
    0.14477192,
    0.25154072,
    0.3373604,
    0.45450795,
    0.53441167,
    0.6008431,
    0.62754464,
    0.6406618,
    0.6073806,
    0.55598694,
    0.5101167,
    0.5095466,
    0.5406314,
    0.5005604,
    0.47417197,
    0.4576856,
    0.44716668,
    0.51430374,
    0.6198342,
    0.63784397,
    0.43150276,
    0.17515111,
    -0.079798646,
    -0.3156405,
    -0.43733022,
    -0.54350084,
    -0.6428162,
    -0.7294537,
    -0.7262573,
    -0.74543536,
    -0.7753426,
    -0.82350254,
    -0.9006352,
    -0.9366173,
    -0.9458184,
    -0.98497355,
    -0.99081016,
    -0.9852352,
    -0.9928569,
    -0.9954411,
    -0.98864186,
    -0.9825015,
    -0.95298684,
    -0.89567244,
    -0.790184,
    -0.6698635,
    -0.5667631,
    -0.47422358,
    -0.33685127,
    -0.17954859,
    -0.04524646,
    0.07530303,
    0.20485273,
    0.2726253,
    0.36247313,
    0.4290868,
    0.5061727,
    0.7686086,
    1.0838028,
    1.37187,
    1.4673582,
    1.2541337,
    0.87544286,
    0.4421432,
    0.11493011,
    -0.06452727,
    -0.119257495,
    -0.09508404,
    -0.048096992,
    0.021320486,
    0.102541976,
    0.13685118,
    0.083363965,
    -0.07357888,
    -0.2510175,
    -0.3977452,
    -0.52359384,
    -0.61722684,
    -0.7333463,
    -0.82606804,
    -0.8737934,
    -0.9420894,
    -1.0360775,
    -1.0697652,
    -1.0637417,
    -1.0340308,
    -0.9789454,
    -0.90310717,
    -0.822096,
    -0.7714967,
    -0.6746206,
    -0.6035163,
    -0.5262107,
    -0.46360183,
    -0.37291285,
    -0.29343438,
    -0.20545106,
    -0.15276761,
    -0.09205594,
    -0.017792558,
    0.08511634,
    0.19057202,
    0.27324682,
    0.38185677,
    0.4643353,
    0.51597196,
    0.5747023,
    0.6427226,
    0.652391,
    0.66300344,
    0.76453376,
    0.9297572,
    1.0891861,
    1.1547016,
    1.0630265,
    0.8277595,
    0.5522812,
    0.32800972,
    0.20086198,
    0.103696205,
    0.11504693,
    0.18840376,
    0.18923087,
    0.093322136,
    -0.043924,
    -0.11370129,
    -0.24426036,
    -0.39041325,
    -0.4571858,
    -0.4720179,
    -0.49003702,
    -0.565006,
    -0.6683635,
    -0.7782866,
    -0.87180275,
    -0.8627932,
    -0.7892635,
    -0.7246218,
    -0.6525454,
    -0.61253047,
    -0.56895006,
    -0.48685938,
    -0.39189926,
    -0.34946838,
    -0.2784901,
    -0.17849717,
    -0.1103414,
    -0.047466137,
    0.036068488,
    0.13157068,
    0.20822196,
    0.25888202,
    0.3269957,
    0.42627367,
    0.4801207,
    0.50070995,
    0.5847633,
    0.6673306,
    0.6937331,
    0.71249986,
    0.7491222,
    0.76957595,
    0.75852895,
    0.7931559,
    0.83899343,
    0.8544798,
    0.84081125,
    0.8126984,
    0.68970966,
    0.57068354,
    0.4573772,
    0.42612413,
    0.4408394,
    0.4841348,
    0.50011647,
    0.36567882,
    0.15123935,
    0.025470113,
    -0.07027974,
    -0.1985023,
    -0.3561321,
    -0.37960926,
    -0.3732353,
    -0.39039922,
    -0.39837605,
    -0.3799831,
    -0.33202875,
    -0.26666275,
    -0.29184088,
    -0.2882333,
    -0.21594663,
    -0.13892616,
    -0.12541652,
    -0.07010684,
    -0.050358724,
    -0.014568186,
    0.016077375,
    0.058003567,
    0.06708789,
    0.073634766,
    0.09940172,
    0.17941289,
    0.25506416,
    0.27182156,
    0.3865251,
    0.4684008,
    0.4983127,
    0.5766509,
    0.6432226,
    0.6895788,
    0.7293975,
    0.7699498,
    0.7620618,
    0.80485713,
    0.85109186,
    0.85363865,
    0.8448534,
    0.85845184,
    0.76736563,
    0.71526164,
    0.64544696,
    0.56681424,
    0.485261,
    0.37380517,
    0.30755132,
    0.22202602,
    0.08459764,
    0.018988656,
    -0.026306773,
    -0.035208847,
    -0.09323353,
    -0.22437204,
    -0.2668964,
    -0.33964574,
    -0.3519778,
    -0.37412784,
    -0.45348477,
    -0.5296267,
    -0.5362624,
    -0.56550604,
    -0.5667631,
    -0.50803745,
    -0.46486822,
    -0.39906764,
    -0.30974784,
    -0.21452136,
    -0.17154841,
    -0.09182229,
    -0.006063319,
    0.05569043,
    0.09615398,
    0.12862201,
    0.15362258,
    0.13513151,
    0.08568645,
    0.10830379,
    0.13582778,
    0.13214079,
    0.11991153,
    0.1323651,
    0.08222843,
    0.0838079,
    0.13769232,
    0.19499736,
    0.25090986,
    0.28423306,
    0.32760787,
    0.35568795,
    0.43097004,
    0.46103615,
    0.4876676,
    0.51462615,
    0.51352334,
    0.45190042,
    0.3901233,
    0.3204021,
    0.23061967,
    0.06212983,
    -0.14137015,
    -0.27230304,
    -0.36863706,
    -0.47381234,
    -0.57900167,
    -0.6412227,
    -0.6697093,
    -0.7222105,
    -0.7440241,
    -0.6487042,
    -0.5138086,
    -0.5710062,
    -0.6209325,
    -0.6422414,
    -0.65533984,
    -0.6439798,
    -0.587871,
    -0.6087313,
    -0.6146426,
    -0.56368357,
    -0.5649499,
    -0.55330485,
    -0.5324959,
    -0.5195424,
    -0.42814776,
    -0.38049245,
    -0.30753753,
    -0.22822727,
    -0.16065097,
    -0.037652828,
    0.10545793,
    0.19235244,
    0.1932076,
    0.2624335,
    0.3606974,
    0.38070253,
    0.38076794,
    0.4227222,
    0.5265423,
    0.5769874,
    0.6003805,
    0.60285246,
    0.64510113,
    0.8247688,
    1.0248201,
    1.1595615,
    0.9627813,
    0.60705817,
    0.22599807,
    -0.054475643,
    -0.24582581,
    -0.32797724,
    -0.38926837,
    -0.4433724,
    -0.48370042,
    -0.50493455,
    -0.59664696,
    -0.6861162,
    -0.6974109,
    -0.6711813,
    -0.7236825,
    -0.78453904,
    -0.8533864,
    -0.87082607,
    -0.8700504,
    -0.8799758,
    -0.89404154,
    -0.85849404,
    -0.91700006,
    -0.95989823,
    -0.93969214,
    -0.8662465,
    -0.7476971,
    -0.7014484,
    -0.588072,
    -0.46967208,
    -0.37853447,
    -0.2850323,
    -0.17503914,
    -0.09069609,
    0.038063858,
    0.19248796,
    0.316645,
    0.3657536,
    0.43757766,
    0.5299162,
    0.57031435,
    0.5757911,
    0.7119952,
    0.99611384,
    1.2473065,
    1.4109364,
    1.246699,
    0.9438089,
    0.5739079,
    0.24544711,
    0.05044732,
    -0.1279119,
    -0.22187667,
    -0.23732562,
    -0.19231524,
    -0.122089334,
    -0.1733335,
    -0.24254069,
    -0.29746717,
    -0.36257148,
    -0.41832042,
    -0.45895687,
    -0.54735607,
    -0.59950215,
    -0.66591483,
    -0.7778193,
    -0.86091465,
    -0.97283775,
    -1.0399234,
    -1.0628632,
    -1.0568256,
    -1.0141658,
    -0.9596365,
    -0.8782795,
    -0.83344203,
    -0.7121402,
    -0.5732726,
    -0.47039172,
    -0.3670342,
    -0.27533117,
    -0.18525434,
    -0.102317855,
    0.028077649,
    0.11701427,
    0.17694087,
    0.281733,
    0.37566504,
    0.44264787,
    0.5734546,
    0.6299606,
    0.67968136,
    0.69961643,
    0.7282339,
    0.82275474,
    1.0130348,
    1.1884032,
    1.409212,
    1.421988,
    1.226507,
    0.9357386,
    0.5242759,
    0.25333047,
    0.13781849,
    0.12229476,
    0.129594,
    0.11540676,
    0.07892928,
    0.116832025,
    0.14412236,
    0.11877132,
    0.003670549,
    -0.16848291,
    -0.319921,
    -0.38115603,
    -0.43757787,
    -0.485093,
    -0.5244724,
    -0.52975756,
    -0.5427999,
    -0.5702024,
    -0.5724081,
    -0.533739,
    -0.46620002,
    -0.36671177,
    -0.29972425,
    -0.20717072,
    -0.075709775,
    0.023633623,
    0.12991643,
    0.19424967,
    0.22438589,
    0.2950604,
    0.39659542,
    0.5050231,
    0.5768659,
    0.5887821,
    0.62306327,
    0.6692232,
    0.70502305,
    0.72143936,
    0.74410343,
    0.733449,
    0.75414103,
    0.7726321,
    0.7163317,
    0.7321919,
    0.79105777,
    0.8194603,
    0.7743237,
    0.7150747,
    0.709883,
    0.6965696,
    0.6610688,
    0.6019506,
    0.56131417,
    0.46784472,
    0.35955718,
    0.31921983,
    0.28141055,
    0.18166529,
    0.14681402,
    0.1643004,
    0.09986434,
    -0.057615902,
    -0.15943599,
    -0.19904904,
    -0.26073268,
    -0.3290614,
    -0.36665103,
    -0.3983854,
    -0.4279889,
    -0.4067781,
    -0.40096956,
    -0.4003387,
    -0.38360468,
    -0.4108296,
    -0.37927747,
    -0.3108133,
    -0.26158786,
    -0.17383818,
    -0.12035098,
    -0.07024703,
    -0.0012734894,
    0.018455934,
    0.07341514,
    0.08337799,
    0.075265646,
    0.13822035,
    0.187114,
    0.18307653,
    0.16852947,
    0.23070846,
    0.29325196,
    0.3772959,
    0.46123242,
    0.52458435,
    0.5496503,
    0.5725153,
    0.6247082,
    0.67936826,
    0.67063445,
    0.5915999,
    0.5586973,
    0.54054266,
    0.56508994,
    0.6195679,
    0.6605688,
    0.6003805,
    0.40899757,
    0.16333309,
    -0.024960948,
    -0.19904904,
    -0.33691204,
    -0.41995132,
    -0.50133634,
    -0.53393525,
    -0.50176626,
    -0.44791925,
    -0.43810594,
    -0.36068827,
    -0.35759008,
    -0.42555425,
    -0.424386,
    -0.4113109,
    -0.42721784,
    -0.44044244,
    -0.4629523,
    -0.4349283,
    -0.42972723,
    -0.3596275,
    -0.25544286,
    -0.26147103,
    -0.24772772,
    -0.18954882,
    -0.15760884,
    -0.12411742,
    -0.100551456,
    -0.09667286,
    -0.031596612,
    0.0644383,
    0.10907951,
    0.19082437,
    0.2622045,
    0.3126449,
    0.4312738,
    0.52435535,
    0.5993197,
    0.5968991,
    0.56292164,
    0.5585851,
    0.5361547,
    0.4786861,
    0.533192,
    0.57041246,
    0.5174206,
    0.3930486,
    0.30730367,
    0.2293346,
    0.18908134,
    0.27289635,
    0.27452722,
    0.1305099,
    -0.056971025,
    -0.24277434,
    -0.45290065,
    -0.58763736,
    -0.65201735,
    -0.7273181,
    -0.7673144,
    -0.77437997,
    -0.77943146,
    -0.8138201,
    -0.82485306,
    -0.77854824,
    -0.78514653,
    -0.83399814,
    -0.8640082,
    -0.8288345,
    -0.8763776,
    -0.8929294,
    -0.9075559,
    -0.9091167,
    -0.8596202,
    -0.79763746,
    -0.74556154,
    -0.7136122,
    -0.659639,
    -0.5459729,
    -0.3885721,
    -0.27152267,
    -0.1359448,
    -0.006465197,
    0.14513174,
    0.27096638,
    0.3652676,
    0.44068053,
    0.54249597,
    0.83801675,
    1.1301216,
    1.4625777,
    1.5327849,
    1.2498906,
    0.8544938,
    0.46450818,
    0.14730468,
    -0.043115567,
    -0.11651444,
    -0.13179517,
    -0.02604041,
    0.11581331,
    0.1608564,
    0.14167371,
    0.021231698,
    -0.17025399,
    -0.38052517,
    -0.49949518,
    -0.56941736,
    -0.66555035,
    -0.76532364,
    -0.88076556,
    -0.94233704,
    -1.0210258,
    -1.1142615,
    -1.165417,
    -1.1495568,
    -1.0898311,
    -1.042545,
    -1.0021188,
    -0.9101541,
    -0.83766645,
    -0.76181424,
    -0.6560408,
    -0.6007218,
    -0.5291828,
    -0.40645567,
    -0.29352316,
    -0.16656698,
    -0.07150407,
    0.010109949,
    0.10830379,
    0.23289542,
    0.3232573,
    0.37077707,
    0.45497993,
    0.5254395,
    0.5710433,
    0.66710633,
    0.7131588,
    0.8746251,
    1.1200466,
    1.3137892,
    1.2564281,
    1.0689707,
    0.7926886,
    0.51509815,
    0.3121776,
    0.20497422,
    0.18355317,
    0.19035707,
    0.16808085,
    0.23630205,
    0.2513164,
    0.19177766,
    0.059026957,
    -0.056611206,
    -0.17216992,
    -0.3028645,
    -0.4181335,
    -0.48239198,
    -0.5705809,
    -0.6779198,
    -0.75718796,
    -0.80890876,
    -0.82026887,
    -0.82570356,
    -0.81418926,
    -0.7344211,
    -0.6275682,
    -0.5306875,
    -0.3889693,
    -0.26721883,
    -0.19648823,
    -0.13855232,
    -0.10581794,
    0.003670549,
    0.11839748,
    0.21934839,
    0.26089606,
    0.2858499,
    0.31921983,
    0.3041821,
    0.31755155,
    0.38530076,
    0.45855477,
    0.51369625,
    0.51920104,
    0.46965316,
    0.4978781,
    0.5416548,
    0.5342014,
    0.58328193,
    0.63322234,
    0.6783776,
    0.72579926,
    0.6693821,
    0.6134088,
    0.58379596,
    0.5734546,
    0.5346593,
    0.48057866,
    0.39753938,
    0.29649502,
    0.18682896,
    0.036661956,
    -0.086537115,
    -0.1253324,
    -0.13090262,
    -0.12925772,
    -0.15842661,
    -0.15457138,
    -0.024334764,
    0.080891944,
    0.0847939,
    -0.07960238,
    -0.33514562,
    -0.4304235,
    -0.46159244,
    -0.5070047,
    -0.4949951,
    -0.43767133,
    -0.40339017,
    -0.32114065,
    -0.25435403,
    -0.1951097,
    -0.10073838,
    0.02712903,
    0.13291183,
    0.22250734,
    0.3097056,
    0.36287034,
    0.3942262,
    0.46049407,
    0.52085525,
    0.51807016,
    0.5100793,
    0.49487337,
    0.5167477,
    0.4678821,
    0.4344047,
    0.41983426,
    0.38748774,
    0.4135818,
    0.51393455,
    0.6058432,
    0.5732303,
    0.5672208,
    0.5739453,
    0.60374504,
    0.6024366,
    0.5868848,
    0.58781475,
    0.54564553,
    0.5264769,
    0.45895198,
    0.41546035,
    0.33873427,
    0.31016353,
    0.26617655,
    0.19544597,
    0.06917205,
    -0.09069609,
    -0.21729244,
    -0.30712628,
    -0.3941236,
    -0.49076602,
    -0.6034462,
    -0.6464144,
    -0.6532276,
    -0.6600736,
    -0.65232575,
    -0.55788904,
    -0.537711,
    -0.54312235,
    -0.48003212,
    -0.37857187,
    -0.26248974,
    -0.17638497,
    -0.123953864,
    -0.027507735,
    0.019876527,
    0.05062022,
    0.11018234,
    0.18317465,
    0.23463845,
    0.2735412,
    0.340253,
    0.40306285,
    0.45558742,
    0.4573398,
    0.42286238,
    0.41426873,
    0.43161023,
    0.4292644,
    0.41005367,
    0.37758097,
    0.33838847,
    0.2614288,
    0.21304451,
    0.19437118,
    0.14892621,
    0.14308964,
    0.14097743,
    0.012778234,
    -0.08020987,
    -0.07916779,
    -0.10769648,
    -0.17952523,
    -0.21736722,
    -0.19824529,
    -0.16552024,
    -0.13242602,
    -0.19498353,
    -0.23754525,
    -0.14799647,
    -0.16401553,
    -0.3551788,
    -0.5255986,
]
print(np.array(w1).shape)

vb_pprint(get_vibration_pct(w1))